In [ ]:
ALTER SESSION SET TIMEZONE = 'Europe/London';

In [ ]:
try:
    import pandas as pd
    import requests
    import datetime
    import time
    
    from snowflake.snowpark.context import get_active_session
    session = get_active_session()
    
    new_relic_personal_access_token = session.sql("SELECT prod.raw.new_relic_secrets()").collect()[0][0]
    ACCOUNT_ID = 1801997
    
except:
    import pandas as pd
    import requests
    import datetime
    import time
    import os
    from dotenv import load_dotenv
    load_dotenv()
    new_relic_personal_access_token=os.getenv('new_relic_personal_access_token')
    ACCOUNT_ID = 1801997



In [ ]:
class NewRelic:
    def __init__(self):
        self.new_relic_personal_access_token = new_relic_personal_access_token
        self.ACCOUNT_ID = ACCOUNT_ID
        self.headers = {'Content-Type': 'application/json', 'API-Key': self.new_relic_personal_access_token}
        self.API_URL = 'https://api.newrelic.com/graphql'
        
        # Oblicz zakres: poprzedni dzień
        self.today = datetime.date.today()
        
        self.yesterday = self.today - datetime.timedelta(days=1)
        self.start_time = self.yesterday.isoformat()
        self.end_time = self.today.isoformat()
        
        self.current_date = datetime.datetime.now().date()
        
        self.nrql_percentage_query = "SELECT percentage(count(*), WHERE result = 'SUCCESS') FROM SyntheticCheck where monitorName LIKE '%E2E%' and monitorName not like '%uat%' and monitorName not like '%staging%' and monitorName not like '%dev%'and monitorName not like '%training%'  and monitorName not like '%sand%' and monitorName  not like '%test%' and monitorName not like '%sc%' and monitorName not like '%204%' and monitorName not like '%automation%' and monitorName not like '%template%' and monitorName not like '%del%'  and monitorName not like '%maverick%' and monitorName not like '%ctms%' and monitorName not like '%sarep%'  and monitorName not like '%neuron%' and monitorName not like '%sre%' and monitorName not like '%principiabiopharma%' and monitorName not like '%gsk%'and monitorName not like '%acceleron%' and monitorName not like '%immunity%' and monitorName not like '%zs%'   and monitorName not like '%eisai%' and monitorName not like '%canary%' and monitorName not like '%saas%' and monitorName not like '%zeb%' limit 1000 \
        SINCE 30 days ago"

        #vision monthly
        
        self.overall_succ_rate_of_vis_us = "SELECT (filter(UNIQUECOUNT(CorrelationId), WHERE `message` LIKE 'ResponseHandlerApiFunction processed a request for DocumentId:%' AND cluster_name = 'aksshared-sharedprod-aks-use' AND namespace_name = 'phlexvision-prod') * 1.0 / filter(UNIQUECOUNT(CorrelationId), WHERE `message` LIKE 'DocumentProcess API received a request for DocumentId:%' AND cluster_name = 'aksshared-sharedprod-aks-use' AND namespace_name = 'phlexvision-prod')) * 100 AS 'Success rate of Vision' FROM Log \
        SINCE last month UNTIL this month"
        
        self.overall_succ_rate_of_vis_eun = "SELECT (filter(UNIQUECOUNT(CorrelationId), WHERE `message` LIKE 'ResponseHandlerApiFunction processed a request for DocumentId:%' AND cluster_name = 'aksshared-sharedprod-aks-eun' AND namespace_name = 'phlexvision-prod') * 1.0 / filter(UNIQUECOUNT(CorrelationId), WHERE `message` LIKE 'DocumentProcess API received a request for DocumentId:%' AND cluster_name = 'aksshared-sharedprod-aks-eun' AND namespace_name = 'phlexvision-prod')) * 100 AS 'Success rate of Vision' FROM Log \
        SINCE last month UNTIL this month"

        self.total_doc_entered_vision_us = "SELECT uniqueCount(CorrelationId) as 'Documents' FROM Log WHERE `message` like 'DocumentProcess API received a request for DocumentId:%' and cluster_name = 'aksshared-sharedprod-aks-use' and namespace_name = 'phlexvision-prod' and applicationName = 'PhlexVision.DocumentProcess.Api' \
        since last month until this month"
        
        self.total_doc_entered_vision_eun = "SELECT uniqueCount(CorrelationId) as 'Documents' FROM Log WHERE `message` like 'DocumentProcess API received a request for DocumentId:%' and cluster_name = 'aksshared-sharedprod-aks-eun' and namespace_name = 'phlexvision-prod' and applicationName = 'PhlexVision.DocumentProcess.Api' \
        SINCE last month UNTIL this month"

        self.stuck_docu_count_eun = "SELECT count(documentid) AS Documents FROM sqlJobs WHERE jobName = 'stuckDocsInfo' AND (('Production' = 'Production' AND hostname like '%production%') OR ('Production' = 'Non-Production' AND hostname not like '%production%')) AND site in ('zs-test', 'biolinerx-dev', 'ccs-test', 'menarini', 'securabio', 'georgemedicines', 'ccs', 'cromsource-test', 'tgtherapeutics-test', 'zs-dev', 'expresseu-ayr', 'intelliatx', 'dcri-training', 'pacira', 'akouos-dev', 'delivery-sandbox', 'alk-test', 'sitero-dev', 'sumitomosunovion', 'neuron-215-us', 'zebedee', 'lung', 'intercept', 'dots-dev', 'ccs-dev', 'zs', 'mhicc', 'delmr', 'pra-test', 'tgtherapeutics', 'drreddys-test', 'pra-dev', 'galapagos', 'oncopeptides-dev', 'ucl-dev', 'devops-clinical-produs', 'aerogen', 'express-dev', 'dots', 'alexion', 'expresseu-template', 'grailbio', 'acceleron-dev', 'fhic', 'oncopeptides-test', 'devops-clinical-prodeu', 'eisai', 'inozyme', 'oracle-sandbox', 'freseniuskabi-test', 'cromsource-dev', 'freseniuskabi', 'inozyme-dev', 'unitedtherapeutics', 'del217', 'express', 'del218', 'ucl-test', 'dcri-sandbox', 'opstraining', 'pra-uat', 'acceleron-test', 'anavex', 'drreddys', 'expresseu-test', 'delivery-uat', 'purdue2', 'sunovion', 'pra', 'ucl', 'obipharma', 'alk', 'delmr217', 'neuron2-215-eu', 'eisai-test', 'oncologie-test', 'oncopeptides', 'obipharma-test', 'dots-test', 'indivior', 'mundipharma-dev', 'sareptatherapeutics', 'sitero-test', 'fhic-test', 'blueprint', 'aimmune-dev', 'acceleron', 'oncologie', 'expresseu', 'thirdpole', 'aimmune', 'biolinerx-test', 'opstraining-dev', 'freseniuskabi-dev', 'expresseu-dev', 'pacira-dev', 'tmcpharma-test', 'mundipharma', 'inozyme-test', 'trizell', 'karyopharm', 'fhic-dev', 'eisai-dev', 'mmv', 'tgtherapeutics-dev', 'dcri', 'vwave', 'express-test', 'galapagos-dev', 'thirdpole-test', 'turningpoint', 'mhicc-test', 'alk-dev', 'obipharma-dev', 'biolinerx', 'akouos', 'indivior-dev', 'pra-sandbox', 'express-template', 'intelliatx-test', 'cromsource', 'levicept', 'tmcpharma', 'rho', 'akouos-test', 'aimmune-test', 'sitero') AND minutes_ago > 30 AND ProcessingStatus = 'OCR Processing' \
        SINCE last month UNTIL this month LIMIT MAX"
        
        self.stuck_docu_count_us = "SELECT count(documentid) AS Documents FROM sqlJobs WHERE jobName = 'stuckDocsInfo' AND (('Production' = 'Production' AND hostname like '%production%') OR ('Production' = 'Non-Production' AND hostname not like '%production%')) AND site in ('zs-test', 'biolinerx-dev', 'ccs-test', 'menarini', 'securabio', 'georgemedicines', 'ccs', 'cromsource-test', 'tgtherapeutics-test', 'zs-dev', 'expresseu-ayr', 'intelliatx', 'dcri-training', 'pacira', 'akouos-dev', 'delivery-sandbox', 'alk-test', 'sitero-dev', 'sumitomosunovion', 'neuron-215-us', 'zebedee', 'lung', 'intercept', 'dots-dev', 'ccs-dev', 'zs', 'mhicc', 'delmr', 'pra-test', 'tgtherapeutics', 'drreddys-test', 'pra-dev', 'galapagos', 'oncopeptides-dev', 'ucl-dev', 'devops-clinical-produs', 'aerogen', 'express-dev', 'dots', 'alexion', 'expresseu-template', 'grailbio', 'acceleron-dev', 'fhic', 'oncopeptides-test', 'devops-clinical-prodeu', 'eisai', 'inozyme', 'oracle-sandbox', 'freseniuskabi-test', 'cromsource-dev', 'freseniuskabi', 'inozyme-dev', 'unitedtherapeutics', 'del217', 'express', 'del218', 'ucl-test', 'dcri-sandbox', 'opstraining', 'pra-uat', 'acceleron-test', 'anavex', 'drreddys', 'expresseu-test', 'delivery-uat', 'purdue2', 'sunovion', 'pra', 'ucl', 'obipharma', 'alk', 'delmr217', 'neuron2-215-eu', 'eisai-test', 'oncologie-test', 'oncopeptides', 'obipharma-test', 'dots-test', 'indivior', 'mundipharma-dev', 'sareptatherapeutics', 'sitero-test', 'fhic-test', 'blueprint', 'aimmune-dev', 'acceleron', 'oncologie', 'expresseu', 'thirdpole', 'aimmune', 'biolinerx-test', 'opstraining-dev', 'freseniuskabi-dev', 'expresseu-dev', 'pacira-dev', 'tmcpharma-test', 'mundipharma', 'inozyme-test', 'trizell', 'karyopharm', 'fhic-dev', 'eisai-dev', 'mmv', 'tgtherapeutics-dev', 'dcri', 'vwave', 'express-test', 'galapagos-dev', 'thirdpole-test', 'turningpoint', 'mhicc-test', 'alk-dev', 'obipharma-dev', 'biolinerx', 'akouos', 'indivior-dev', 'pra-sandbox', 'express-template', 'intelliatx-test', 'cromsource', 'levicept', 'tmcpharma', 'rho', 'akouos-test', 'aimmune-test', 'sitero') AND minutes_ago > 30 AND ProcessingStatus = 'OCR Processing' \
        SINCE last month UNTIL this month LIMIT MAX"

        #vision daily
        self.avg_time_successful_docu_us = f"SELECT SUM(duration) / COUNT(duration) as 'Average Duration (in seconds)' FROM (SELECT filter(MIN(timestamp), WHERE `message` like 'SuccessCallback to % endpoint' and cluster_name = 'aksshared-sharedprod-aks-use' and namespace_name = 'phlexvision-prod' and applicationName = 'PhlexVision.ResponseHandler.Api') / 1000 - filter(MIN(timestamp), WHERE `message` like 'DocumentProcess API received a request for DocumentId: %, File Extension: %, Priority: %' and cluster_name = 'aksshared-sharedprod-aks-use' and namespace_name = 'phlexvision-prod') / 1000 as duration FROM Log WHERE `Duration (in seconds)` != '' FACET CorrelationId LIMIT MAX) WHERE duration > 0 \
        SINCE yesterday until today"
        
        self.avg_time_successful_docu_eun = f"SELECT SUM(duration) / COUNT(duration) as 'Average Duration (in seconds)' FROM (SELECT filter(MIN(timestamp), WHERE `message` like 'SuccessCallback to % endpoint' and cluster_name = 'aksshared-sharedprod-aks-eun' and namespace_name = 'phlexvision-prod' and applicationName = 'PhlexVision.ResponseHandler.Api') / 1000 - filter(MIN(timestamp), WHERE `message` like 'DocumentProcess API received a request for DocumentId: %, File Extension: %, Priority: %' and cluster_name = 'aksshared-sharedprod-aks-eun' and namespace_name = 'phlexvision-prod') / 1000 as duration FROM Log WHERE `Duration (in seconds)` != '' FACET CorrelationId LIMIT MAX) WHERE duration > 0 \
        SINCE yesterday until today"

        # neuron daily
        self.overall_success_rate_neuron_us = "SELECT (filter(UNIQUECOUNT(CorrelationId), WHERE `message` LIKE '%MLOpsTrigger: Posted processed data for ReferenceID: % to % at: %' AND cluster_name = 'aksshared-sharedprod-aks-use' AND namespace_name = 'neuroncore-prod') * 1.0 / filter(UNIQUECOUNT(CorrelationId), WHERE `message` LIKE '%AxonNeuron: Started posting data to%queue for ReferenceID:%' AND cluster_name = 'aksshared-sharedprod-aks-use' AND namespace_name = 'neuroncore-prod')) * 100 AS 'Success rate of Neuron' FROM Log \
        SINCE last month UNTIL this month"

        self.overall_success_rate_neuron_eun = "SELECT (filter(UNIQUECOUNT(CorrelationId), WHERE `message` LIKE '%MLOpsTrigger: Posted processed data for ReferenceID: % to % at: %' AND cluster_name = 'aksshared-sharedprod-aks-use' AND namespace_name = 'neuroncore-prod') * 1.0 / filter(UNIQUECOUNT(CorrelationId), WHERE `message` LIKE '%AxonNeuron: Started posting data to%queue for ReferenceID:%' AND cluster_name = 'aksshared-sharedprod-aks-use' AND namespace_name = 'neuroncore-prod')) * 100 AS 'Success rate of Neuron' FROM Log \
        SINCE last month UNTIL this month"

        self.avg_neuron_end_to_end_duration_us = "SELECT SUM(duration) / COUNT(duration) as 'Overall Duration in Neuron (in seconds)' FROM (SELECT filter(MIN(timestamp), WHERE `message` like '%MLOpsTrigger: Completed Processing data for ReferenceID:%' and cluster_name = 'aksshared-sharedprod-aks-use' AND namespace_name = 'neuroncore-prod') / 1000 - filter(MIN(timestamp), WHERE `message` like '%AxonNeuron: Started posting data to % queue for ReferenceID: %' and cluster_name = 'aksshared-sharedprod-aks-use' AND namespace_name = 'neuroncore-prod') / 1000 as duration FROM Log WHERE `Duration (in seconds)` != '' FACET CorrelationId LIMIT MAX) WHERE duration > 0 \
        SINCE last month UNTIL this month"

        self.avg_neuron_end_to_end_duration_eun = "SELECT SUM(duration) / COUNT(duration) as 'Overall Duration in Neuron (in seconds)' FROM (SELECT filter(MIN(timestamp), WHERE `message` like '%MLOpsTrigger: Completed Processing data for ReferenceID:%' and cluster_name = 'aksshared-sharedprod-aks-use' AND namespace_name = 'neuroncore-prod') / 1000 - filter(MIN(timestamp), WHERE `message` like '%AxonNeuron: Started posting data to % queue for ReferenceID: %' and cluster_name = 'aksshared-sharedprod-aks-use' AND namespace_name = 'neuroncore-prod') / 1000 as duration FROM Log WHERE `Duration (in seconds)` != '' FACET CorrelationId LIMIT MAX) WHERE duration > 0 \
        SINCE last month UNTIL this month"

        self.documents_stuck_in_ai_all = "SELECT count(*) as 'AI' FROM stuckDocsInfo WHERE site in ('acacia', 'advil', 'advil219', 'aerogen', 'aerogen-test', 'ajdemo', 'ajtest', 'akouos', 'akouos-test', 'alexion', 'alk', 'anavex', 'archer', 'archer-sponsor', 'arfiauto', 'arfiauto2110', 'artemis', 'autocapybara', 'awawa', 'badb-4', 'bagcpaired', 'bemusponsor', 'bemusponsor-test', 'bemusponsor2110', 'biolinerx', 'blueprint', 'blueprint-dev', 'blueprint-test', 'bobby', 'bonsaiauto2', 'brandon', 'canary', 'ccs', 'cheetah', 'circe', 'clinical-staging', 'clinical-testeu', 'clinical-testus', 'clonedlivetestpaired', 'cromsource', 'dcri', 'dcri-dev', 'dcri-sandbox', 'dcri-test', 'dcri-training', 'del2110', 'del219', 'delivery-sandbox', 'delivery21', 'delivery21saas', 'delmr', 'delmr-test', 'delmr217', 'delmr217-dev', 'delmr217-test', 'delmra', 'delmra-test', 'delmrb', 'dev', 'devops-single-cluster', 'devsecops-dast', 'devsecops-testing', 'do-mysql-percona', 'do-mysql-percona-2', 'dots', 'dots-test', 'drreddys', 'eisai', 'expertservice', 'express', 'expresseu', 'fhic', 'freseniuskabi', 'fsp2110', 'fyndemo', 'galapagos', 'georgemedicines', 'georgemedicines-dev', 'gigantsponsor', 'gloop', 'gold210', 'golddataupdatetest', 'grailbio', 'hackerone', 'hulk', 'hulk2', 'importval170', 'indivior', 'inozyme', 'intelliatx', 'intelliatx-test', 'intercept', 'jakedemo', 'jkrota', 'jkrota-test', 'jokaper', 'jokarep', 'jokatpaas', 'jynfake', 'karyopharm', 'karyopharm-test', 'kazetest', 'klon', 'klon-test', 'krosrep', 'krosreports', 'kw2299', 'lemur-forever', 'lemur-reloaded', 'levicept', 'lifetestsite2110', 'lifetestsite2110-test', 'livetestsite2025', 'livetestsite2025-test', 'livetestsite3-test', 'lublinauto21-10-v1', 'lung', 'mabxience', 'mabxience-test', 'markustmf2131', 'marvel', 'marvel-2', 'marvel-2110', 'marvel-test', 'marvel3', 'massrollout', 'massrolloutgroup2', 'massrolloutgroup2-test', 'maszauto4', 'maszlive2', 'maszlive2-test', 'mavericknew', 'menarini', 'mhicc', 'mhicc-test', 'minotaur', 'miru', 'mmv', 'momzauto1', 'momzauto5', 'momzauto6', 'mundipharma', 'naboo', 'nesstmf', 'neuron-215-us', 'neuron2-215-eu', 'neuron219', 'neurontest01', 'newcreate4', 'newtest', 'nonpairedinstance2110', 'nuerontest02', 'oak', 'obipharma', 'oncopeptides', 'pacira', 'pacira-test', 'passrota', 'pra', 'pra-dev', 'pra-sandbox', 'pra-test', 'pra-uat', 'prdpit-1726-secondary-test', 'prdpit-1748-primary', 'prdpit-1748-primary-test', 'prdpit-1748-secondary', 'prdpit-1748-secondary-test', 'prdpit-1812', 'prdpit-1812-test', 'prdpit-1812dev', 'prdpit853', 'prdpxv-55588', 'processingv2', 'purdue2', 'pw-auto-bulkenv-2111-02', 'pw-auto-env-sampling-2111-02', 'pw-auto-env1-2111-15', 'pw-auto-env2-2110-1', 'pw-auto-env2-21101-29', 'pw-auto-env2-2111-10', 'pw-auto-pq-2', 'pw-auto-reports-2111-01', 'pw-auto-sponsor-2110-5', 'ref-sc', 'reportsdemo', 'reportsdescriptionsfix', 'rho', 'rho-dev', 'rho-test', 'saas2110', 'sareptatherapeutics', 'securabio', 'sharondemo', 'sitero', 'slclust', 'sre-test', 'sumitomosunovion', 'sunovion', 'testeqc', 'testgolddata', 'testsitesingle', 'tgtherapeutics', 'tmcpharma', 'training001', 'trizell', 'tyko21112', 'ucl', 'unitedtherapeutics', 'unitedtherapeutics-test', 'visiontest217', 'visiontestint', 'vwave', 'wedelaut-samp-2111', 'westbound', 'wobopit', 'wronskiauto50', 'wronskiauto50test', 'zebedee', 'zurek') AND (('Production' = 'Production' AND environment like 'production%') OR ('Production' = 'Non-Production' AND environment not like 'production%')) AND ProcessingStatus in ('AI Processing', 'AI Processing (Legacy)') AND minutes_ago > 30 \
        SINCE last month UNTIL this month"

    def get_nrql_percentage(self):
        time.sleep(15)
        graphql_query = f"""
            {{
              actor {{
                account(id: {self.ACCOUNT_ID}) {{
                  nrql(query: "{self.nrql_percentage_query}") {{
                    results
                  }}
                }}
              }}
            }}
            """
            
        response = requests.post(self.API_URL, json={'query': graphql_query}, headers=self.headers)
        print(response.json())
        percentage = response.json().get('data', {}) \
                                      .get('actor', {}) \
                                      .get('account', {}) \
                                      .get('nrql', {}) \
                                      .get('results', [{}])[0] \
                                      .get('percentage', 0.0)
            
        new_relic_data = pd.DataFrame([{'percentage': percentage, 'timestamp_date': self.today}])
        new_relic_data['percentage'] = new_relic_data['percentage'].astype(float)
            
        return new_relic_data
        # return response.json()
        
    def get_overall_succ_rate_of_vis_us(self):
        time.sleep(15)
        osr_graphql_query = f"""
        {{
          actor {{
            account(id: {self.ACCOUNT_ID}) {{
                  nrql(query: "{self.overall_succ_rate_of_vis_us}") {{
                results
              }}
            }}
          }}
        }}
        """
        
        response = requests.post(self.API_URL, json={'query': osr_graphql_query}, headers=self.headers)
        print(response.json())
        success_rate_vision_us = response.json().get('data', {}) \
                                  .get('actor', {}) \
                                  .get('account', {}) \
                                  .get('nrql', {}) \
                                  .get('results', [{}])[0] \
                                  .get('Success rate of Vision', 0.0)
        
        success_rate_vision_us_data = pd.DataFrame([{ 'metrics_name':'Success rate of Vision US', 'value': success_rate_vision_us, 'timestamp_date': self.today}])
        success_rate_vision_us_data['value'] = success_rate_vision_us_data['value'].astype(float)
        
        # TEMP_NEWRELIC = 'TEMP_NEWRELIC'
        
        # session.write_pandas(new_relic_data,
        #                          table_name=TEMP_NEWRELIC,
        #                          auto_create_table=True,
        #                          overwrite=True,
        #                          table_type="temporary")
        return success_rate_vision_us_data
        
    def get_overall_succ_rate_of_vis_eun(self):
        time.sleep(15)
        osreun_graphql_query = f"""
        {{
          actor {{
            account(id: {self.ACCOUNT_ID}) {{
                  nrql(query: "{self.overall_succ_rate_of_vis_eun}") {{
                results
              }}
            }}
          }}
        }}
        """
        
        response = requests.post(self.API_URL, json={'query': osreun_graphql_query}, headers=self.headers)
        print(response.json())
        success_rate_vision_eun = response.json().get('data', {}) \
                                  .get('actor', {}) \
                                  .get('account', {}) \
                                  .get('nrql', {}) \
                                  .get('results', [{}])[0] \
                                  .get('Success rate of Vision', 0.0)
        
        success_rate_vision_eun_data = pd.DataFrame([{ 'metrics_name':'Success rate of Vision EUN', 'value': success_rate_vision_eun, 'timestamp_date': self.today}])
        success_rate_vision_eun_data['value'] = success_rate_vision_eun_data['value'].astype(float)
        
        # TEMP_NEWRELIC = 'TEMP_NEWRELIC'
        
        # session.write_pandas(new_relic_data,
        #                          table_name=TEMP_NEWRELIC,
        #                          auto_create_table=True,
        #                          overwrite=True,
        #                          table_type="temporary")
        return success_rate_vision_eun_data

        
    def get_documents_us_data(self):
        time.sleep(15)
        dusd_graphql_query = f"""
        {{
          actor {{
            account(id: {self.ACCOUNT_ID}) {{
                  nrql(query: "{self.stuck_docu_count_us}") {{
                results
              }}
            }}
          }}
        }}
        """
        
        response = requests.post(self.API_URL, json={'query': dusd_graphql_query}, headers=self.headers)
        print(response.json())
        documents_us = response.json().get('data', {}) \
                                  .get('actor', {}) \
                                  .get('account', {}) \
                                  .get('nrql', {}) \
                                  .get('results', [{}])[0] \
                                  .get('Documents', 0.0)
        
        documents_us_data = pd.DataFrame([{ 'metrics_name':'Documents US', 'value': documents_us, 'timestamp_date': self.today}])
        documents_us_data['value'] = documents_us_data['value'].astype(float)
        
        # TEMP_NEWRELIC = 'TEMP_NEWRELIC'
        
        # session.write_pandas(new_relic_data,
        #                          table_name=TEMP_NEWRELIC,
        #                          auto_create_table=True,
        #                          overwrite=True,
        #                          table_type="temporary")
        return documents_us_data
    
    def get_documents_eun_data(self):
        time.sleep(15)
        deund_graphql_query = f"""
        {{
          actor {{
            account(id: {self.ACCOUNT_ID}) {{
                  nrql(query: "{self.stuck_docu_count_eun}") {{
                results
              }}
            }}
          }}
        }}
        """
        
        response = requests.post(self.API_URL, json={'query': deund_graphql_query}, headers=self.headers)
        print(response.json())
        documents_eun = response.json().get('data', {}) \
                                  .get('actor', {}) \
                                  .get('account', {}) \
                                  .get('nrql', {}) \
                                  .get('results', [{}])[0] \
                                  .get('Documents', 0.0)
        
        documents_eun_data = pd.DataFrame([{ 'metrics_name':'Documents EUN', 'value': documents_eun, 'timestamp_date': self.today}])
        documents_eun_data['value'] = documents_eun_data['value'].astype(float)
        
        # TEMP_NEWRELIC = 'TEMP_NEWRELIC'
        
        # session.write_pandas(new_relic_data,|
        #                          table_name=TEMP_NEWRELIC,
        #                          auto_create_table=True,
        #                          overwrite=True,
        #                          table_type="temporary")
        return documents_eun_data

    #vision daily
    def get_avg_duraiton_us(self):
        time.sleep(15)      
        graphql_query = f"""
        {{
          actor {{
            account(id: {self.ACCOUNT_ID}) {{
                  nrql(query: "{self.avg_time_successful_docu_us}") {{
                results
              }}
            }}
          }}
        }}
        """
        
        response = requests.post(self.API_URL, json={'query': graphql_query}, headers=self.headers)
        print(response.json())
        avg_duraiton_us = response.json().get('data', {}) \
                                  .get('actor', {}) \
                                  .get('account', {}) \
                                  .get('nrql', {}) \
                                  .get('results', [{}])[0] \
                                  .get('Average Duration (in seconds)', 0.0)
        
        avg_duraiton_us_data = pd.DataFrame([{ 'metrics_name':'Average Duration (in seconds) US', 'value': avg_duraiton_us, 'timestamp_date': self.start_time}])
        # avg_duraiton_us_data['value'] = avg_duraiton_us_data['value'].astype(float)
        
        # # TEMP_NEWRELIC = 'TEMP_NEWRELIC'
        
        # # session.write_pandas(new_relic_data,
        # #                          table_name=TEMP_NEWRELIC,
        # #                          auto_create_table=True,
        # #                          overwrite=True,
        # #                          table_type="temporary")
        
        return avg_duraiton_us_data

    def get_avg_duraiton_eun(self):
        error_occurred = True  # Start assuming we'll get an error
        
        while error_occurred:
            time.sleep(15)  # Your original delay
            
            graphql_query = f"""
            {{
              actor {{
                account(id: {self.ACCOUNT_ID}) {{
                      nrql(query: "{self.avg_time_successful_docu_eun}") {{
                    results
                  }}
                }}
              }}
            }}
            """
            
            response = requests.post(self.API_URL, json={'query': graphql_query}, headers=self.headers)
            response_data = response.json()
            print(response_data)
            
            # Check if we got an error - if 'nrql' is None or if there are errors
            if (response_data.get('errors') or 
                response_data.get('data', {}).get('actor', {}).get('account', {}).get('nrql') is None):
                
                error_occurred = True  # Keep the loop going
                print("Got error, retrying in 15 seconds...")
                
            else:
                # Success! We got data
                error_occurred = False  # This will exit the loop
                
                avg_duraiton_eun = response_data.get('data', {}) \
                                          .get('actor', {}) \
                                          .get('account', {}) \
                                          .get('nrql', {}) \
                                          .get('results', [{}])[0] \
                                          .get('Average Duration (in seconds)', 0.0)
                
                avg_duraiton_eun_data = pd.DataFrame([{
                    'metrics_name':'Average Duration (in seconds) EUN', 
                    'value': avg_duraiton_eun, 
                    'timestamp_date': self.start_time
                }])
                
                print(f"Success! Got value: {avg_duraiton_eun}")
                return avg_duraiton_eun_data


####################################### Neuron
        #######################################
            #######################################
        
    def get_success_of_neuron_us(self):
            time.sleep(15)
            graphql_query = f"""
            {{
              actor {{
                account(id: {self.ACCOUNT_ID}) {{
                      nrql(query: "{self.overall_success_rate_neuron_us}") {{
                    results
                  }}
                }}
              }}
            }}
            """
            
            
            response = requests.post(self.API_URL, json={'query': graphql_query}, headers=self.headers)
            print(response.json())
            success_of_neuron_us = response.json().get('data', {}) \
                                      .get('actor', {}) \
                                      .get('account', {}) \
                                      .get('nrql', {}) \
                                      .get('results', [{}])[0] \
                                      .get('Success rate of Neuron', 0.0)
            
            success_of_neuron_us_data = pd.DataFrame([{ 'metrics_name':'Success rate of Neuron US', 'value': success_of_neuron_us, 'timestamp_date': self.current_date}])
            success_of_neuron_us_data['value'] = success_of_neuron_us_data['value'].astype(float)
            
            # TEMP_NEWRELIC = 'TEMP_NEWRELIC'
            
            # session.write_pandas(new_relic_data,
            #                          table_name=TEMP_NEWRELIC,
            #                          auto_create_table=True,
            #                          overwrite=True,
            #                          table_type="temporary")
            return success_of_neuron_us_data
            # response.json()


    def get_success_of_neuron_eun(self):
        time.sleep(15)
        graphql_query = f"""
        {{
          actor {{
            account(id: {self.ACCOUNT_ID}) {{
                  nrql(query: "{self.overall_success_rate_neuron_eun}") {{
                results
              }}
            }}
          }}
        }}
        """
        
        response = requests.post(self.API_URL, json={'query': graphql_query}, headers=self.headers)
        print(response.json())
        success_of_neuron_eun = response.json().get('data', {}) \
                                  .get('actor', {}) \
                                  .get('account', {}) \
                                  .get('nrql', {}) \
                                  .get('results', [{}])[0] \
                                  .get('Success rate of Neuron', 0.0)
        
        success_of_neuron_eun_data = pd.DataFrame([{ 'metrics_name':'Success rate of Neuron EUN', 'value': success_of_neuron_eun, 'timestamp_date': self.current_date}])
        success_of_neuron_eun_data['value'] = success_of_neuron_eun_data['value'].astype(float)
        
        # TEMP_NEWRELIC = 'TEMP_NEWRELIC'
        
        # session.write_pandas(new_relic_data,
        #                          table_name=TEMP_NEWRELIC,
        #                          auto_create_table=True,
        #                          overwrite=True,
        #                          table_type="temporary")
        return success_of_neuron_eun_data
        # response.json()

    def get_avg_neuron_e2e_duration_us(self):
        time.sleep(15)
        graphql_query = f"""
        {{
          actor {{
            account(id: {self.ACCOUNT_ID}) {{
                  nrql(query: "{self.avg_neuron_end_to_end_duration_us}") {{
                results
              }}
            }}
          }}
        }}
        """
        
        
        response = requests.post(self.API_URL, json={'query': graphql_query}, headers=self.headers)
        print(response.json())
        avg_neuron_e2e_duration_us = response.json().get('data', {}) \
                                  .get('actor', {}) \
                                  .get('account', {}) \
                                  .get('nrql', {}) \
                                  .get('results', [{}])[0] \
                                  .get('Overall Duration in Neuron (in seconds)', 0.0)
        
        avg_neuron_e2e_duration_us_data = pd.DataFrame([{ 'metrics_name':'Overall Duration in Neuron (in seconds) US', 'value': avg_neuron_e2e_duration_us, 'timestamp_date': self.current_date}])
        avg_neuron_e2e_duration_us_data['value'] = avg_neuron_e2e_duration_us_data['value'].astype(float)
        
        # TEMP_NEWRELIC = 'TEMP_NEWRELIC'
        
        # session.write_pandas(new_relic_data,
        #                          table_name=TEMP_NEWRELIC,
        #                          auto_create_table=True,
        #                          overwrite=True,
        #                          table_type="temporary")
        return avg_neuron_e2e_duration_us_data
        # response.json()

    def get_avg_neuron_e2e_duration_eun(self):
            time.sleep(15)
            graphql_query = f"""
            {{
              actor {{
                account(id: {self.ACCOUNT_ID}) {{
                      nrql(query: "{self.avg_neuron_end_to_end_duration_eun}") {{
                    results
                  }}
                }}
              }}
            }}
            """
            
            response = requests.post(self.API_URL, json={'query': graphql_query}, headers=self.headers)
            print(response.json())
            avg_neuron_e2e_duration_eun = response.json().get('data', {}) \
                                      .get('actor', {}) \
                                      .get('account', {}) \
                                      .get('nrql', {}) \
                                      .get('results', [{}])[0] \
                                      .get('Overall Duration in Neuron (in seconds)', 0.0)
            
            avg_neuron_e2e_duration_eun_data = pd.DataFrame([{ 'metrics_name':'Overall Duration in Neuron (in seconds) EUN', 'value': avg_neuron_e2e_duration_eun, 'timestamp_date': self.current_date}])
            avg_neuron_e2e_duration_eun_data['value'] = avg_neuron_e2e_duration_eun_data['value'].astype(float)
            
            # TEMP_NEWRELIC = 'TEMP_NEWRELIC'
            
            # session.write_pandas(new_relic_data,
            #                          table_name=TEMP_NEWRELIC,
            #                          auto_create_table=True,
            #                          overwrite=True,
            #                          table_type="temporary")
            return avg_neuron_e2e_duration_eun_data
        # response.json()

    def get_documents_stuck_in_ai(self):
        time.sleep(15)
        graphql_query = f"""
        {{
          actor {{
            account(id: {self.ACCOUNT_ID}) {{
                  nrql(query: "{self.documents_stuck_in_ai_all}") {{
                results
              }}
            }}
          }}
        }}
        """
        
        response = requests.post(self.API_URL, json={'query': graphql_query}, headers=self.headers)
        print(response.json())
        documents_stuck_in_ai_value_all = response.json().get('data', {}) \
                                  .get('actor', {}) \
                                  .get('account', {}) \
                                  .get('nrql', {}) \
                                  .get('results', [{}])[0] \
                                  .get('AI', 0.0)
        
        documents_stuck_in_ai_all_data = pd.DataFrame([{ 'metrics_name':'Documents Stuck in AI (Both Prod regions) ALL', 'value': documents_stuck_in_ai_value_all, 'timestamp_date': self.current_date}])
        documents_stuck_in_ai_all_data['value'] = documents_stuck_in_ai_all_data['value'].astype(float)
        
        # TEMP_NEWRELIC = 'TEMP_NEWRELIC'
        
        # session.write_pandas(new_relic_data,
        #                          table_name=TEMP_NEWRELIC,
        #                          auto_create_table=True,
        #                          overwrite=True,
        #                          table_type="temporary")
        return documents_stuck_in_ai_all_data
        # response.json()


# success_rate_vision_eun
# Przykład użycia:
# new_relic_personal_access_token = "twój_token"
# ACCOUNT_ID = "twoje_account_id"
api = NewRelic()

# Percentage Success

In [ ]:
nrql = api.get_nrql_percentage()

In [ ]:
TEMP_PERCENTAGE_SUCCESS = 'TEMP_PERCENTAGE_SUCCESS'
session.write_pandas(nrql,
                         table_name=TEMP_PERCENTAGE_SUCCESS,
                         auto_create_table=True,
                         overwrite=True,
                         table_type="temporary")
nrql

In [ ]:
-- -- create or replace table prod.silver.newrelic_api_call
-- -- (percentage float,
-- -- timestamp_date date);

insert into prod.raw.newrelic_api_call
select
"percentage",
"timestamp_date"
from TEMP_PERCENTAGE_SUCCESS;



In [ ]:
-- create or replace table prod.silver.newrelic_api_call as
insert overwrite into prod.silver.newrelic_api_call 
with cte as (select *,
row_number() OVER (partition by timestamp_date ORDER BY timestamp_date desc) as rank_asc,
from prod.raw.newrelic_api_call
order by timestamp_date desc)
select percentage, timestamp_date from cte
where rank_asc = 1

In [ ]:
select * from prod.silver.newrelic_api_call 

# vision monthly part

In [ ]:
success_rate_vision_us_data_df = api.get_overall_succ_rate_of_vis_us()
success_rate_vision_us_data_df

In [ ]:
success_rate_vision_eun_data_df = api.get_overall_succ_rate_of_vis_eun()
success_rate_vision_eun_data_df

In [ ]:
get_documents_us_data_df = api.get_documents_us_data()
get_documents_us_data_df

In [ ]:
get_documents_eun_data_df = api.get_documents_eun_data()
get_documents_eun_data_df

In [ ]:
df_monthly = pd.concat([success_rate_vision_us_data_df, success_rate_vision_eun_data_df, get_documents_us_data_df, get_documents_eun_data_df]).reset_index().drop(columns='index')
# df_monthly['timestamp_date'] = pd.to_datetime(df_monthly['timestamp_date'])
df_monthly

In [ ]:
TEMP_VISION_MONTHLY = 'TEMP_VISION_MONTHLY'

session.write_pandas(df_monthly,
                         table_name=TEMP_VISION_MONTHLY,
                         auto_create_table=True,
                         overwrite=True,
                         table_type="temporary")

In [ ]:
insert into prod.raw.newrelic_vision_monthly
select * from TEMP_VISION_MONTHLY

In [ ]:

create or replace table  prod.silver.newrelic_vision_monthly as

WITH ranked_data AS (
    SELECT "metrics_name",
           "value",
           month("timestamp_date") AS month,
           year("timestamp_date") AS year,
           "timestamp_date",
           row_number() OVER (PARTITION BY "metrics_name" ORDER BY "timestamp_date" DESC) AS rank
    FROM prod.raw.newrelic_vision_monthly
)
SELECT "metrics_name",
       "value",
       month,
       year,
       month - 1 as covered_month,
       year as covered_year
       -- "timestamp_date"
FROM ranked_data
WHERE rank = 1
ORDER BY "metrics_name";

select * from prod.silver.newrelic_vision_monthly


# vision daily part

In [ ]:
avg_duraiton_us_data = api.get_avg_duraiton_us()
avg_duraiton_us_data

In [ ]:
avg_duraiton_eun_data = api.get_avg_duraiton_eun()
avg_duraiton_eun_data

In [ ]:
df_daily = pd.concat([avg_duraiton_us_data, avg_duraiton_eun_data]).reset_index().drop(columns='index')
df_monthly['timestamp_date'] = pd.to_datetime(df_monthly['timestamp_date'])
df_daily

In [ ]:
TEMP_VISION_DAILY = 'TEMP_VISION_DAILY'

session.write_pandas(df_daily,
                         table_name=TEMP_VISION_DAILY,
                         auto_create_table=True,
                         overwrite=True,
                         table_type="temporary")

In [ ]:
insert into prod.raw.newrelic_vision_daily
select * from TEMP_VISION_DAILY

In [ ]:
select * from prod.raw.newrelic_vision_daily

In [ ]:
create or replace table prod.silver.newrelic_vision as

select 
metrics_name, 
avg
    (value) as avg_value, 
-- month(timestamp_date) as month,  
-- year(timestamp_date) as year,  
-- timestamp_date,
--added 01 to the date to keep date formatting in power bi, 
date
    (case when 
    length(month(timestamp_date)) = 1
    then concat(year(timestamp_date), '-','0',month(timestamp_date)-1, '-01')
    else
    concat(year(timestamp_date), '-',month(timestamp_date), '-01')
    end) as covered_month,
from prod.raw.newrelic_vision_daily
group by metrics_name, covered_month;

select * from prod.silver.newrelic_vision

# Neuron part

In [ ]:
get_success_of_neuron_us = api.get_success_of_neuron_us()
get_success_of_neuron_us

In [ ]:
get_success_of_neuron_eun = api.get_success_of_neuron_eun()
get_success_of_neuron_eun

In [ ]:
get_avg_neuron_e2e_duration_us = api.get_avg_neuron_e2e_duration_us()
get_avg_neuron_e2e_duration_us

In [ ]:
get_avg_neuron_e2e_duration_eun = api.get_avg_neuron_e2e_duration_eun()
get_avg_neuron_e2e_duration_eun

In [ ]:
get_documents_stuck_in_ai = api.get_documents_stuck_in_ai()
get_documents_stuck_in_ai

In [ ]:
neuron_monthly = pd.concat([get_success_of_neuron_us, get_success_of_neuron_eun, get_avg_neuron_e2e_duration_us, get_avg_neuron_e2e_duration_eun, get_documents_stuck_in_ai]).reset_index().drop(columns='index')
neuron_monthly

In [ ]:
TEMP_NEURON_MONTHLY = 'TEMP_NEURON_MONTHLY'

session.write_pandas(neuron_monthly,
                         table_name=TEMP_NEURON_MONTHLY,
                         auto_create_table=True,
                         overwrite=True,
                         table_type="temporary")

In [ ]:
create or replace table prod.raw.neuron_monthly as
select * from TEMP_NEURON_MONTHLY;

In [ ]:
create or replace table prod.silver.neuron_monthly as

with cte as (select *,
date
    (case when 
    length(month("timestamp_date")) = 1
    then concat(year("timestamp_date"), '-','0',month("timestamp_date")-1, '-01')
    else
    concat(year("timestamp_date"), '-',month("timestamp_date"), '-01')
    end) as covered_month,
from prod.raw.neuron_monthly
group by "metrics_name", covered_month, "value","timestamp_date")
select * from cte;

select * from prod.silver.neuron_monthly
where day("timestamp_date") = 01
-- row_number() over (partition by "metrics_name" order by covered_month desc) as rank,